<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/get_all_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import subprocess
import re
from google.colab import drive  # Import for Google Drive

def mount_drive(drive_path):
    """
    Mounts Google Drive to the specified path.

    Args:
        drive_path (str): The path where Google Drive should be mounted.

    Returns:
        bool: True if the mounting was successful, False otherwise.
    """
    try:
        drive.mount(drive_path)
        return True
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return False

def find_files_with_pattern(root_dir, pattern):
    """
    Recursively finds all files in a directory whose name matches a given pattern.

    Args:
        root_dir (str): The root directory to start the search from.
        pattern (str): The pattern to match (e.g., "result").

    Returns:
        list: A list of paths to the files that match the pattern.
    """
    matching_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for dirname in dirnames:
            if pattern in dirname:
                matching_files.append(os.path.join(dirpath, dirname))
    return matching_files

def run_loftr_ranking(folder_path):
    """
    Runs the loftr_ranking.py script on a specified folder.

    Args:
        folder_path (str): The path to the folder to process.

    Returns:
        tuple: (highest_ranking_image_name, all_results_text) if successful, (None, None) otherwise.
    """
    script_path = os.path.join(drive_path, "realfill_repo", "tools", "loftr_ranking.py") # Corrected path
    if not os.path.exists(script_path):
        print(f"Error: loftr_ranking.py not found at {script_path}")
        return None, None

    try:
        # Construct the command to run the script.
        command = ['python', script_path, folder_path]
        process = subprocess.run(command, check=True, capture_output=True, text=True)
        output_text = process.stdout

        # Extract the highest ranking image.
        match = re.search(r"Highest ranking image: (.*)", output_text)
        if match:
            highest_ranking_image_name = match.group(1).strip()
            return highest_ranking_image_name, output_text
        else:
            print(f"Warning: No highest ranking image found in output for folder {folder_path}")
            return None, output_text

    except subprocess.CalledProcessError as e:
        print(f"Error running loftr_ranking.py on {folder_path}: {e}")
        print(f"Command: {e.cmd}")
        print(f"Output: {e.output}")
        return None, None
    except FileNotFoundError:
        print("Error: python command not found.  Ensure Python is installed and in your system's PATH.")
        return None, None

def process_folders(folders):
    """
    Processes a list of folders, running loftr_ranking.py on each and collecting results.

    Args:
        folders (list): A list of folder paths to process.

    Returns:
        dict: A dictionary where keys are folder names and values are the names of the highest-ranking images.
              Returns an empty dictionary if no results are found.
    """
    results = {}
    for folder in folders:
        highest_ranking_image, all_results = run_loftr_ranking(folder) # Capture all_results
        if highest_ranking_image:
            folder_name = os.path.basename(folder)  # Get just the folder name
            results[folder_name] = highest_ranking_image
        else:
            folder_name = os.path.basename(folder)
            print(f"No results for folder: {folder}")
    return results

if __name__ == "__main__":
    # Define the  destination directory for mounting.  MODIFY THIS PATH AS NEEDED.
    drive_path = "/content/drive/MyDrive"  #  The root of your Google Drive.  You may need to adjust this.
    repo_path = os.path.join(drive_path, "RealFill") # Path within Drive

    # Mount Google Drive.
    if not mount_drive(drive_path):
        print("Failed to mount Google Drive. Exiting.")
        exit()

    # Find all folders containing "result".
    result_folders = find_files_with_pattern(repo_path, "result")
    if not result_folders:
        print("No folders containing 'result' found in the Drive path.")
        exit()

    # Process the found folders.
    results = process_folders(result_folders)

    # Print the results.
    if results:
        print("Results:")
        for folder, image_name in results.items():
            print(f"{folder}: {image_name}")
    else:
        print("No results found.")
